# Answers

Import standard libraries and the class `Loans_Portfolio` from `portfolio_model` which is our custom class for this project.

In [19]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from case.portfolio_model import Loans_Portfolio

The following are just optional configurations.

In [19]:
mystyle = 'style/pastel-light.mplstyle'
plt.style.use(mystyle)
%config InlineBackend.figure_format='retina'
pd.options.mode.copy_on_write = True 
pd.options.display.float_format = "{:,.2f}".format

Create the `Loans_Portfolio` object, passin the cob date as well as the spreadsheet location.
We assume that the spreadsheet is in the same format as the one provided.

In [20]:
cob = datetime.date(2022, 12, 31)  # 31/12/2022
ss_path = 'data/2024_Strat_Casestudy.xlsx'
portfolio = Loans_Portfolio(cob_date=cob, data_path=ss_path)

Question 1: Consolidate the Static and Historic Data provided into a single pandas dataframe to allow for easier data exploration and facilitate future analysis.

The dataframe should be of the format where there is a row per loan per calendar month.

In [21]:
portfolio.consolidate_data()

The consolidated data can be seen as follows

In [22]:
consolidated_data = portfolio.data

In [23]:
consolidated_data.head()

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin
0,1,2020-12-31,"150,876.00",NaN,NaN,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
1,1,2021-01-31,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
2,1,2021-02-28,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
3,1,2021-03-31,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04
4,1,2021-04-30,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,0.02,0.04


In [24]:
consolidated_data.describe()

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin
count,"42,611.00",42611,"42,611.00","37,170.00","37,170.00",42611,42611,42611,"42,611.00","42,611.00","42,611.00","42,611.00"
mean,866.32,2021-05-20 17:39:24.657013504,"135,323.46",370.39,"1,694.07",2019-10-09 23:02:16.781582336,2021-03-07 14:39:33.527962112,2021-10-09 09:32:48.590270208,"151,748.79",1.29,0.03,0.03
min,1.00,2016-01-31 00:00:00,0.00,0.00,0.00,2016-01-31 00:00:00,2020-12-31 00:00:00,2018-01-31 00:00:00,"50,244.00",1.00,0.02,0.02
25%,441.00,2020-08-31 00:00:00,"86,145.00",226.09,220.72,2018-09-30 00:00:00,2020-12-31 00:00:00,2020-09-30 00:00:00,"102,895.00",1.00,0.02,0.02
50%,889.00,2021-08-31 00:00:00,"140,448.00",339.61,338.02,2019-11-30 00:00:00,2020-12-31 00:00:00,2021-11-30 00:00:00,"150,368.00",1.00,0.03,0.04
75%,"1,300.00",2022-05-31 00:00:00,"193,355.00",469.95,472.15,2020-12-31 00:00:00,2020-12-31 00:00:00,2022-12-31 00:00:00,"201,128.00",2.00,0.03,0.04
max,"1,630.00",2022-12-31 00:00:00,"249,978.00","1,536.21","248,504.78",2022-12-31 00:00:00,2022-12-31 00:00:00,2024-12-31 00:00:00,"249,978.00",2.00,0.08,0.04
std,484.40,NaN,"71,605.64",199.44,"15,051.62",NaN,NaN,NaN,"57,376.29",0.45,0.01,0.01


Question 2: Please add the following calculated columns to the dataframe (plus any others you found helpful). 

Dynamic Columns (These vary by loan and by calendar month):

- current_balance: The current balance outstanding for each loan and month.
- seasoning: The integer number of months since the loan was originated at each month.
- n_missed_payments: number of missed payments in a row.
- prepaid_in_month: a flag indicating that the borrower prepaid in a given month.
- default_in_month: a flag indicating that the borrower defaulted in a given month.
- recovery_in_month: a flag indicating that a recovery has been made post-default in a given month.
- is_recovery_payment: a flag indicating whether the associated payment has been made post-default.
- time_to_reversion: The integer number of months until the laon reverts. This is negative if the - loan is before reversion and 0 at the month of reversion.
- is_post_seller_purchsae_date: Is this time period after the seller purchased this loan.

 Static Columns (These vary by loan but are the same for each calendar month):

- postdefault_recoveries: The cumulative recoveries post-default.
- prepayment_date: The date that the loan prepays (or nan if it does not).
- date_of_default: the date that the loan defaults (or nan if it does not).
- date_of_recovery: the date that a recovery is made on the loan, post-default.
- exposure_at_default: the current balance of the loan outstanding at default.
- recovery_percent: the postdefault_recoveries as a percentage of the exposure at default.

In [25]:
portfolio.create_enriched_data_portfolio()

/Users/dialidsantiago/Git/strat-case-study-draft/case/portfolio_model.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  enriched_data = pd.concat([df for df in extended_sub_datas if not df.empty])


In [26]:
portfolio.enriched_data

,loan_id,level_1,Balance,Payment_Due,Payment_Made,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,...,recovery_in_month,recovery_cumsum,is_recovery_payment,time_to_reversion,is_post_seller_purchsae_date,postdefault_recoveries,prepayment_date,date_of_default,exposure_at_default,recovery_percent
0,1,2020-12-31,"150,876.00",NaN,0.00,2020-12-31,2020-12-31,2022-12-31,150876,1,...,0.00,0.00,False,-24,False,0.00,2021-08-31,NaT,NaN,NaN
1,1,2021-01-31,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,...,0.00,0.00,False,-23,True,0.00,2021-08-31,NaT,NaN,NaN
2,1,2021-02-28,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,...,0.00,0.00,False,-22,True,0.00,2021-08-31,NaT,NaN,NaN
3,1,2021-03-31,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,...,0.00,0.00,False,-21,True,0.00,2021-08-31,NaT,NaN,NaN
4,1,2021-04-30,"150,876.00",249.96,249.96,2020-12-31,2020-12-31,2022-12-31,150876,1,...,0.00,0.00,False,-20,True,0.00,2021-08-31,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42606,1630,2022-08-31,"81,951.00",325.20,325.20,2022-05-31,2022-05-31,2024-05-31,81951,2,...,0.00,0.00,False,-21,True,0.00,NaT,NaT,NaN,NaN
42607,1630,2022-09-30,"81,951.00",325.20,325.20,2022-05-31,2022-05-31,2024-05-31,81951,2,...,0.00,0.00,False,-20,True,0.00,NaT,NaT,NaN,NaN
42608,1630,2022-10-31,"81,951.00",325.20,325.20,2022-05-31,2022-05-31,2024-05-31,81951,2,...,0.00,0.00,False,-19,True,0.00,NaT,NaT,NaN,NaN
42609,1630,2022-11-30,"81,951.00",325.20,325.20,2022-05-31,2022-05-31,2024-05-31,81951,2,...,0.00,0.00,False,-18,True,0.00,NaT,NaT,NaN,NaN


Question 3: Create a function that returns Prepayment Curves ("CPR") for the portfolio. The function should by default return CPR as a pandas series with an index of `seasoning` for the whole portfolio.  However, your function should be able to take a list of`pivots' which are a list of column names, whereby the function will then return a dataframe with each column being the CPR for that unique value of pivot.

In [27]:
portfolio.construct_portfolio_cpr()

,smm,cpr
0,0.00,0.02
1,0.00,0.01
2,0.00,0.00
3,0.00,0.01
4,0.00,0.03
...,...,...
79,0.04,0.42
80,0.00,0.00
81,0.00,0.00
82,0.00,0.00


Question 4: Create a similar function for Default Curves ("CDR") 

In [28]:
portfolio.construct_portfolio_cdr()

,mdr,cdr
0,0.00,0.00
1,0.00,0.00
2,0.00,0.02
3,0.00,0.01
4,0.00,0.02
...,...,...
79,0.00,0.00
80,0.00,0.00
81,0.00,0.00
82,0.00,0.00


Question 6: Using your function,  calculate Prepayment and Default Curves (vs. Time to Reversion); split by Product

In [29]:
portfolio.construct_recovery_curve()

,recovery_curve
-50,0.00
-49,0.00
-48,0.00
-47,0.00
-46,0.00
...,...
-67,0.00
-66,0.00
-65,0.00
-64,0.00


Question 7: Using your function, construct Recovery Curves split by year of default ("Default Vintage")

In [30]:
portfolio.construct_recovery_curve(pivots=['year_of_default'])

,recovery_curve_year_of_default_2022,recovery_curve_year_of_default_2021
-50,0.00,0.00
-49,0.00,0.00
-48,0.00,0.00
-47,0.00,0.00
-46,0.00,0.00
...,...,...
-67,0.00,NaN
-66,0.00,NaN
-65,0.00,NaN
-64,0.00,NaN


Question 8: Decompose your Recovery Curve in Q7 into two stage calibration:

(1) Probability of Recovery by Time Since Default and
(2) Recovery as a % of EAD

To express the expected recovery as the sum of recoveries by a given time after default using the law of total probability, we introduce the following notation


- $T$ = Time since default
- $P(T = t) $ = Probability of a recovery occurring at time $t$
- $R_t$ = Recovery rate as a % of EAD at time $t$
- $EAD$ = Exposure at Default
- $E[R_t]$ = Expected recovery rate as a percentage of EAD at time $t$


Using the law of total probability, the expected recovery by a given time $T$ can be written as:

$$
E[\text{Recovery}] = \sum_{t=0}^{T} P(T = t) \cdot E[\text{Recovery} \mid T = t]
$$

Where $$E[\text{Recovery} \mid T = t] $$ is the expected recovery amount given that the recovery occurs at time $t$.



Given that the recovery at time $t$ is $R_t \times EAD$, the expected recovery amount given a recovery occurs at time $t$ is:

$$
E[\text{Recovery} \mid T = t] = E[R_t] \cdot EAD
$$


Combining these, the overall expected recovery by a given time $T$ from default is:

$$
E[\text{Recovery by time T}] = EAD \cdot \sum_{t=0}^{T} P(T = t) \cdot E[R_t]
$$



Question 9: Build a Python Cashflow Model that allows the user to forecasts expected cashflows on a loan-by-loan basis using input CPR / CDR and Recovery Curves. A simple excel example has been given on the "Simple Mortgage Model" tab

In [32]:
from case.forecast_model import Loan, Model_Inputs, ForecastModel

CPR = portfolio.construct_portfolio_cpr(index='time_to_reversion')
CDR = portfolio.construct_portfolio_cdr(index='time_to_reversion')
n = len(CDR)

model_inputs = Model_Inputs({
    'CPR': CPR.cpr.reset_index(drop=True),
    'CDR': CDR.cdr.reset_index(drop=True),
    'months_for_reversion': CPR.index,
    'boe_rates_forecast': np.full(n, 0.02) # fixed for now

})

loan_data = Loan({
    'Months_Post_Reversion': -22,
    'Seasoning': 2,
    'current_balance': 100000,
    'Fixed Pre-Reversion Rate': 0.0394,
    'Post Reversion Margin': 0.0494,
    'Months to Maturity': 178,
    'repayment_method': 'Interest Only'
})

model = ForecastModel(model_inputs)
forecast_table = model.forecast(loan_data)

In [33]:
forecast_table.head()

,Expected_Opening_Perfornace,Expected Closing Default Balance,Defaults,Expected Balance Post Period Defaults,Expected Balance Pre Period Prepays,Expected Prepayments
0,"100,000.00","99,817.90",0.00,"100,000.00","100,000.00",182.10
1,"99,817.90","99,765.36",0.00,"99,817.90","99,817.90",52.53
2,"99,765.36","99,526.82",202.67,"99,562.70","99,562.70",35.87
3,"99,526.82","99,370.32",55.03,"99,471.79","99,471.79",101.48
4,"99,370.32","98,922.03",164.31,"99,206.01","99,206.01",283.98
